In [1]:
from generation.ontology.event_retriever import EventRetriever
from generation.ontology.similarity_calculator import LocalSemanticSimilarityCalculator
from generation.adaptation.astar import ConstructiveAdaptation
from generation.adaptation.query import Query
from common.utils.loader import load_json_folder, load_txt_folder, data_dir, out_dir, save_raw_folktale, load_json
from generation.ontology.folktale_graph import FolktaleOntology
from loguru import logger
from rdflib import Graph
from dotenv import load_dotenv
from generation.story_generator import generate_story
from common.models.folktale import AnnotatedFolktale
import os


In [2]:
def create_graph(build: bool=False, render_html: bool=False) -> Graph:
	graph = FolktaleOntology()
	if build:
		hierarchies = load_json_folder(f"{data_dir}/hierarchies")
		graph.build(hierarchies)

		examples = load_json_folder(f"{data_dir}/examples/annotated")
		for folktale in examples.values():
			graph.add_folktale(folktale)

		graph.add_imports()
		graph.save()
		if render_html:
			graph.render_html("instances")

	logger.debug(f"Grafo initialized with {len(graph)} triplets.")

	graph.load()

	logger.debug(f"Grafo initialized with {len(graph)} triplets after loading.")

	return graph

In [3]:
graph = create_graph(build=False,
						 render_html=False)
	
event_retriever = EventRetriever(graph)
sim_calculator = LocalSemanticSimilarityCalculator(graph)

2026-01-07 13:04:33.776 | DEBUG    | __main__:create_graph:16 - Grafo initialized with 0 triplets.
2026-01-07 13:04:33.807 | DEBUG    | __main__:create_graph:20 - Grafo initialized with 1112 triplets after loading.


In [4]:
def execute_query(query: str):
	try:
		results = graph.query(query)
		result_list = list(results)
		return result_list
	except Exception as e:
		print("Error in SPARQL query:", e)
		return []

In [5]:
from rdflib import Namespace
from rdflib.namespace import RDF, RDFS

ONT = Namespace("https://rbc.org/ontology/")
RES = Namespace("https://rbc.org/resources/")

WD = Namespace("http://www.wikidata.org/entity/")		# elementos
WDT = Namespace("http://www.wikidata.org/prop/direct/")	# propiedades
SCHEMA = Namespace("https://schema.org/")

SEM = Namespace("http://semanticweb.cs.vu.nl/2009/11/sem/")
REL = Namespace("http://purl.org/vocab/relationship/")
PEARL = Namespace("https://www.gsi.upm.es/ontologies/pearl/")


In [8]:
def get_event_type_name(event_uri: str) -> str | None:
    query = f"""
    PREFIX rdf: <{RDF}>
    PREFIX ont: <{ONT}>

    SELECT ?type
    WHERE {{
        <{event_uri}> rdf:type ?type .
        FILTER(STRSTARTS(STR(?type), STR(ont:)))
    }}
    LIMIT 1
    """
    
    results = execute_query(query)

    if not results:
        return None

    return str(results[0].type).split("/")[-1]


In [6]:
def get_event_type(event_uri: str) -> list[str]:
    query = f"""
    PREFIX rdf: <{RDF}>
    
    SELECT ?type
    WHERE {{
        <{event_uri}> rdf:type ?type .
    }}
    """
    
    results = execute_query(query)
    return [str(row.type) for row in results]


In [9]:
get_event_type_name("https://rbc.org/resources/event/the_hare_and_the_tortoise/challenge_issued")

'Setup'

In [6]:
def get_folktales_by_genre(genre: str) -> list[str]:
	genre_map = {
		"fable": "wd:Q693",
		"fairy_tale": "wd:Q699",
		"legend": "wd:Q44342",
		"myth": "wd:Q12827256",
		"tall_tale": "wd:Q1951220",
	}

	if genre not in genre_map:
		raise ValueError(f"Unsupported genre: {genre}")

	genre_wd = genre_map[genre]

	query = f"""
	PREFIX rdfs: <{RDFS}>
	PREFIX rdf: <{RDF}>
	PREFIX ont: <{ONT}>
	PREFIX wd: <{WD}>

	SELECT ?folktale ?title
	WHERE {{
		?folktale a ont:Folktale ;
		   ont:hasGenre {genre_wd} ;
		   ont:title ?title .
	}}
	"""
	results = execute_query(query)

	return [(str(result.folktale), str(result.title))for result in results] if results else []

In [7]:
get_folktales_by_genre("fable")

[('https://rbc.org/resources/folktale/the_hare_and_the_tortoise',
  'The Hare and The Tortoise'),
 ('https://rbc.org/resources/folktale/the_three_little_pigs',
  'The Three Little Pigs')]

In [8]:
def get_roles_by_type_and_folktale_genre(role_type,genre
) -> list[tuple[str, str, str]]:
	"""
	Returns:
		[(role_uri, role_label, folktale_title), ...]
	"""

	query = f"""
	PREFIX rdfs: <{RDFS}>
	PREFIX rdf: <{RDF}>
	PREFIX rdfs: <{RDFS}>
	PREFIX ont: <{ONT}>
	PREFIX wd: <{WD}>


	SELECT DISTINCT ?agent ?roleLabel ?folktaleTitle
	WHERE {{
		?folktale a ont:Folktale ;
				  ont:hasGenre <{genre}> ;
				  ont:title ?folktaleTitle ;
				  ont:hasEvent ?event .

		?event ont:hasAgent ?agent .
		?agent ont:hasRole ?role .

		?role rdf:type <{role_type}> .
		OPTIONAL {{ ?role rdfs:label ?roleLabel }}
	}}
	"""

	results = execute_query(query)

	return [
		(str(row.agent), str(row.roleLabel), str(row.folktaleTitle))
		for row in results
	]


In [ ]:
def get_place_by_type_and_folktale_genre(place_type,genre
) -> list[tuple[str, str, str]]:

	query = f"""
	PREFIX rdfs: <{RDFS}>
	PREFIX rdf: <{RDF}>
	PREFIX rdfs: <{RDFS}>
	PREFIX ont: <{ONT}>
	PREFIX wd: <{WD}>


	SELECT DISTINCT ?place ?placeLabel ?folktaleTitle
	WHERE {{
		?folktale a ont:Folktale ;
				  ont:hasGenre <{genre}> ;
				  ont:title ?folktaleTitle ;
				  ont:hasEvent ?event .

		?event ont:hasPlace ?place .
		?place rdf:type ont:{place_type} .
		OPTIONAL {{ ?place rdfs:label ?placeLabel }}
	}}
	"""

	results = execute_query(query)

	return [
		(str(row.place), str(row.placeLabel), str(row.folktaleTitle))
		for row in results
	]


In [25]:
results = get_place_by_type_and_folktale_genre(
    place_type="Forest",
    genre="Q693"  # Fable
)

In [26]:
results

[('https://rbc.org/resources/place/the_hare_and_the_tortoise/race_track',
  'Race Track',
  'The Hare and The Tortoise')]

In [10]:
roles = get_roles_by_type_and_folktale_genre(
	role_type=ONT.Hero,
	genre=FolktaleOntology.GENRE_MAP["fable"]
)


In [11]:
roles

[('https://rbc.org/resources/agent/the_hare_and_the_tortoise/tortoise',
  'Hero',
  'The Hare and The Tortoise'),
 ('https://rbc.org/resources/agent/the_three_little_pigs/pig_three',
  'Hero',
  'The Three Little Pigs')]

In [27]:
def get_objects_by_type_and_genre(object_type,genre
) -> list[tuple[str, str, str]]:
	

	query = f"""
	PREFIX rdfs: <{RDFS}>
	PREFIX rdf: <{RDF}>
	PREFIX rdfs: <{RDFS}>
	PREFIX ont: <{ONT}>
	PREFIX wd: <{WD}>


	SELECT DISTINCT ?object ?objectLabel ?folktaleTitle
	WHERE {{
		?folktale a ont:Folktale ;
				  ont:hasGenre <{genre}> ;
				  ont:title ?folktaleTitle ;
				  ont:hasEvent ?event .

		?event ont:hasObject ?object .
		?object rdf:type ont:{object_type} .
		OPTIONAL {{ ?object rdfs:label ?objectLabel }}
	}}
	"""

	results = execute_query(query)

	return [
		(str(row.object), str(row.objectLabel), str(row.folktaleTitle))
		for row in results
	]


In [29]:
get_objects_by_type_and_genre("NaturalObject",FolktaleOntology.GENRE_MAP["fable"])

[('https://rbc.org/resources/object/the_three_little_pigs/straw',
  'Straw',
  'The Three Little Pigs'),
 ('https://rbc.org/resources/object/the_three_little_pigs/stick',
  'Stick',
  'The Three Little Pigs')]

In [12]:
def get_ordered_events_for_agent(agent_uri) -> list[tuple[str, str]]:
    query = f"""
    PREFIX rdfs: <{RDFS}>
    PREFIX ont: <{ONT}>

    SELECT ?event ?label ?eventType (COUNT(?prev) AS ?order)
    WHERE {{
        ?event ont:hasAgent <{agent_uri}> ;
            rdf:type ?eventType .
        OPTIONAL {{ ?event rdfs:label ?label }}

        OPTIONAL {{
            ?prev ont:postEvent+ ?event .
        }}
    }}
    GROUP BY ?event ?label
    ORDER BY ?order
    """
    results = execute_query(query)

    return [
        (str(r.eventType),str(r.event), str(r.label),str(r.order))
        for r in results
    ]


In [13]:
agent_uri = "https://rbc.org/resources/role/the_three_little_pigs/hero"

events = get_ordered_events_for_agent(roles[0][0])

for eventType,event_uri, label,order in events:
    print(eventType)

https://rbc.org/ontology/Setup
https://rbc.org/ontology/Conflict
https://rbc.org/ontology/Setup
https://rbc.org/ontology/Setup
https://rbc.org/ontology/Move
https://rbc.org/ontology/ConnectiveIncident
https://rbc.org/ontology/Victory


In [14]:
events

[('https://rbc.org/ontology/Setup',
  'https://rbc.org/resources/event/the_hare_and_the_tortoise/challenge_issued',
  'Challenge Issued',
  '0'),
 ('https://rbc.org/ontology/Conflict',
  'https://rbc.org/resources/event/the_hare_and_the_tortoise/antagonist_mocking_hero',
  'Antagonist Mocking Hero',
  '1'),
 ('https://rbc.org/ontology/Setup',
  'https://rbc.org/resources/event/the_hare_and_the_tortoise/challenge_accepted',
  'Challenge Accepted',
  '2'),
 ('https://rbc.org/ontology/Setup',
  'https://rbc.org/resources/event/the_hare_and_the_tortoise/race_begins',
  'Race Begins',
  '3'),
 ('https://rbc.org/ontology/Move',
  'https://rbc.org/resources/event/the_hare_and_the_tortoise/hero_steady_progress',
  'Hero Steady Progress',
  '5'),
 ('https://rbc.org/ontology/ConnectiveIncident',
  'https://rbc.org/resources/event/the_hare_and_the_tortoise/hero_passing_antagonist',
  'Hero Passing Antagonist',
  '6'),
 ('https://rbc.org/ontology/Victory',
  'https://rbc.org/resources/event/the_ha

In [15]:
def get_role_classes(event_uri: str):
		query = f"""
		PREFIX rdfs: <{RDFS}>
		PREFIX rdf: <{RDF}>
		PREFIX ont: <{ONT}>

		SELECT ?roleClass (COUNT(?role) AS ?roleCount) (GROUP_CONCAT(DISTINCT ?agent; separator=", ") AS ?agents)
		WHERE {{
			<{event_uri}> ont:hasAgent ?agent .
			?agent ont:hasRole ?role .
			?role rdf:type ?roleClass .
		}}
		GROUP BY ?roleClass
		"""

		results = execute_query(query)

		if results:
			dict_result = {str(result.roleClass).split('/')[-1]: (int(result.roleCount), str(result.agents).split(',')) for result in results}
			return dict_result
		return {}


In [16]:
def get_object_classes(event_uri: str):
    query = f"""
    PREFIX rdfs: <{RDFS}>
    PREFIX rdf: <{RDF}>
    PREFIX ont: <{ONT}>

    SELECT ?objectClass (COUNT(?object) AS ?objectCount) (GROUP_CONCAT(DISTINCT ?object; separator=",") AS ?objects)
    WHERE {{
        <{event_uri}> ont:hasObject ?object .
        ?object rdf:type ?objectClass .
    }}
    GROUP BY ?objectClass
    """

    results = execute_query(query)

    if results:
        dict_result = {str(result.objectClass).split('/')[-1]: (int(result.objectCount),str(result.objects).split(',') ) for result in results}
        return dict_result

    return {}


In [17]:
get_object_classes("https://rbc.org/resources/event/the_three_little_pigs/villain_fails_sturdy_house")

{}

In [18]:
d["HeroFamily"][1][0]

NameError: name 'd' is not defined